In [ ]:
# Import Libraries
from arango import ArangoClient
import networkx as nx

# Load graph into ArangoDB
def load_graph_to_arangodb(graph, db_name="graph_db", graph_name="ai_incidents_graph"):
    client = ArangoClient(hosts="http://localhost:8529")
    sys_db = client.db("_system", username="root", password="")
    if not sys_db.has_database(db_name):
        sys_db.create_database(db_name)
    db = client.db(db_name, username="root", password="")
    if not db.has_graph(graph_name):
        graph_collection = db.create_graph(graph_name)
    graph_collection = db.graph(graph_name)
    if not graph_collection.has_vertex_collection("nodes"):
        graph_collection.create_vertex_collection("nodes")
    if not graph_collection.has_edge_collection("edges"):
        graph_collection.create_edge_definition(
            edge_collection="edges",
            from_vertex_collections=["nodes"],
            to_vertex_collections=["nodes"]
        )
    nodes_collection = db.collection("nodes")
    for node in graph.nodes():
        nodes_collection.insert({"_key": str(node)})
    edges_collection = db.collection("edges")
    for edge in graph.edges(data=True):
        edges_collection.insert({
            "_from": f"nodes/{edge[0]}",
            "_to": f"nodes/{edge[1]}",
            "weight": edge[2].get("weight", 1)
        })
    print(f"Graph '{graph_name}' loaded into ArangoDB.")

G = nx.read_graphml("data/graph/graph.graphml")
load_graph_to_arangodb(G)